<a href="https://colab.research.google.com/github/graciekureel1/ProjectAiden/blob/main/Aiden_ML_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install flirt
!pip install neurokit2
!pip install seaborn
!pip install scipy
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import neurokit2 as nk
import scipy.signal
from scipy import stats
import seaborn as sns

import flirt.with_
import flirt.reader.empatica
!pip install --upgrade tensorflow
import tensorflow as tf
from sklearn.preprocessing import StandardScaler

In [ ]:
def read_and_process_data(person_name):
    data_path = f'/content/drive/MyDrive/Participants/{person_name.lower()}/'
    hr_df = pd.read_csv(f'{data_path}HR.csv')
    eda_df = flirt.reader.empatica.read_eda_file_into_df(f'{data_path}EDA.csv')
    tags_df = pd.read_csv(f'{data_path}tags.csv', header=None)

    tags_df = pd.to_datetime(tags_df[0], unit='s', utc=True).dt.strftime('%Y-%m-%d %H:%M:%S+00:00')
    water_drinking_timestamp = tags_df.iloc[0]

    exclude_window = 45  # 45 seconds
    eda_df.index = pd.to_datetime(eda_df.index, utc=True)

    dehydration_eda_df = eda_df[eda_df.index < water_drinking_timestamp]
    hydration_eda_df = eda_df[(eda_df.index > (pd.to_datetime(water_drinking_timestamp) + pd.Timedelta(seconds=exclude_window)))]

    heda_features = flirt.get_eda_features(hydration_eda_df['eda'],
                                           window_length=60,
                                           window_step_size=1,
                                           data_frequency=4)
    deda_features = flirt.get_eda_features(dehydration_eda_df['eda'],
                                           window_length=60,
                                           window_step_size=1,
                                           data_frequency=4)

    deda_features = deda_features.drop('phasic_entropy', axis=1)
    deda_features['State'] = 0
    heda_features = heda_features.drop('phasic_entropy', axis=1)
    heda_features['State'] = 1

    combined_df = pd.concat([deda_features, heda_features], axis=0)
    return combined_df

In [ ]:
# List of participant names
participants = ['gracy1', 'gracy2', 'gracy3', 'kabeer', 'ananth', 'sam', 'jahnavi', 'karthika', 'zach']

# Process data for each participant and combine
all_data = pd.DataFrame()
for person in participants:
    participant_data = read_and_process_data(person)

 # Drop NaNs and infinite values before standardization
    participant_data.replace([np.inf, -np.inf], np.nan, inplace=True)
    participant_data.dropna(inplace=True)

    # Extract features and labels
    X_participant = participant_data.iloc[:, :-1]
    y_participant = participant_data.iloc[:, -1]

    # Standardize features
    sc = StandardScaler()
    X_participant = sc.fit_transform(X_participant)

    # Recombine features and labels
    participant_data_combined = pd.DataFrame(X_participant, columns=participant_data.columns[:-1])
    participant_data_combined['State'] = y_participant.values

    # Add to the overall dataset
    all_data = pd.concat([all_data, participant_data_combined], axis=0)

EDA features:   0%|          | 0/682 [00:00<?, ?it/s]

<ipython-input-51-4d58dbbcbc31>:7: UserWarning: phasic_entropy contains more than 5% (actual: 68.91%) nan, inf, or -inf values. We recommend to delete this feature column.
  participant_data = read_and_process_data(person)


EDA features:   0%|          | 0/311 [00:00<?, ?it/s]

<ipython-input-51-4d58dbbcbc31>:7: UserWarning: phasic_entropy contains more than 5% (actual: 72.03%) nan, inf, or -inf values. We recommend to delete this feature column.
  participant_data = read_and_process_data(person)


EDA features:   0%|          | 0/982 [00:00<?, ?it/s]

...

<ipython-input-51-4d58dbbcbc31>:7: UserWarning: phasic_entropy contains more than 5% (actual: 66.49%) nan, inf, or -inf values. We recommend to delete this feature column.
  participant_data = read_and_process_data(person)


EDA features:   0%|          | 0/865 [00:00<?, ?it/s]

<ipython-input-51-4d58dbbcbc31>:7: UserWarning: phasic_entropy contains more than 5% (actual: 69.48%) nan, inf, or -inf values. We recommend to delete this feature column.
  participant_data = read_and_process_data(person)


EDA features:   0%|          | 0/377 [00:00<?, ?it/s]

<ipython-input-51-4d58dbbcbc31>:7: UserWarning: tonic_entropy contains more than 5% (actual: 7.96%) nan, inf, or -inf values. We recommend to delete this feature column.
  participant_data = read_and_process_data(person)
<ipython-input-51-4d58dbbcbc31>:7: UserWarning: phasic_entropy contains more than 5% (actual: 63.4%) nan, inf, or -inf values. We recommend to delete this feature column.
  participant_data = read_and_process_data(person)


In [ ]:

cleaned_shuffled_df = cleaned_shuffled_df.replace([np.inf, -np.inf], np.nan)
cleaned_shuffled_df.dropna(inplace=True)
X = cleaned_shuffled_df.iloc[:, :-1].values
y = cleaned_shuffled_df.iloc[:, -1].values
from sklearn.model_selection import train_test_split
y

array(['H', 'H', 'H', ..., 'H', 'H', 'H'], dtype=object)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)
X_train = X_train.astype('float32')
y_train = y_train.astype('float32')
# Define a simple neural network model
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),  # Added comma here
    tf.keras.layers.Dense(1, activation='sigmoid')  # Use 'softmax' for multi-class classification
])


# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy}")

# Make predictions
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5).astype("int32")  # Convert probabilities to class labels

# Display predictions
print(np.concatenate((y_pred, y_test.reshape(len(y_test), 1)), axis=1))

ValueError: ignored